In [2]:
from transformers import pipeline
import polars as pl

/Users/adrianochs/mambaforge/envs/hugging_face/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
classifier = pipeline("sentiment-analysis") # , model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
classifier("I've been waiting for a HuggingFace course my whole life.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598051905632019}]

In [4]:
df_fed = pl.read_csv("data/1_df_text.csv") # 2_df_text_clean
df_fed.head()


[{'label': 'NEGATIVE', 'score': 0.9994264841079712},
 {'label': 'NEGATIVE', 'score': 0.607053816318512},
 {'label': 'NEGATIVE', 'score': 0.9975897073745728},
 {'label': 'NEGATIVE', 'score': 0.9923612475395203},
 {'label': 'NEGATIVE', 'score': 0.508060097694397},
 {'label': 'NEGATIVE', 'score': 0.9930084943771362},
 {'label': 'POSITIVE', 'score': 0.9953112006187439},
 {'label': 'NEGATIVE', 'score': 0.9828432202339172},
 {'label': 'NEGATIVE', 'score': 0.9956054091453552}]

In [ ]:
statement_column = df_fed.select(pl.col("statement")).to_series().to_list()
pos_neg_classification = classifier(statement_column[1:10])
pos_neg_classification

In [7]:
statement_column[8]

'treasury cash balances at both commercial banks and federal reserve banks had been reduced to very low levels prior to the mid september tax date, and in the period september 5-16 the treasury had temporarily financed some of its cash needs through sales of special short-term certificates of indebtedness to the federal reserve. the volume of such certificates reached a 16-year high of $1.1 billion on september 10,1/ but the treasury was able to redeem all outstanding certificates by september 17 and subsequently to rebuild its cash balances to a substantial level.'

In [30]:
zero_shot_classifier = pipeline("zero-shot-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
econ_classification = zero_shot_classifier(
    statement_column[1:10],
    candidate_labels=["gdp", "unemployment", "inflation", "other"],
)
econ_classification

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


[{'sequence': ' average price as measure by the gnp deflator be estimate to have advance substantially',
  'labels': ['unemployment', 'inflation', 'gdp', 'other'],
  'scores': [0.5807117223739624,
   0.2618597149848938,
   0.08929827809333801,
   0.06813033670186996]},
 {'sequence': ' staff projections suggest that real gnp would grow more slowly in the fourth quarter and that it might change little in the first half of 1970',
  'labels': ['inflation', 'other', 'gdp', 'unemployment'],
  'scores': [0.2750270962715149,
   0.2721661329269409,
   0.2506636381149292,
   0.2021431177854538]},
 {'sequence': ' with pressure on resources expect to ease over that period some moderation in the rate of price advance be projected',
  'labels': ['other', 'gdp', 'inflation', 'unemployment'],
  'scores': [0.2721790373325348,
   0.2700256407260895,
   0.22974678874015808,
   0.22804847359657288]},
 {'sequence': 'a number of monthly measure of economic_activity have weaken recently',
  'labels': ['unemp